In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', None)          # Adjust width to avoid line wrapping
pd.set_option('display.max_colwidth', None)   # Display full content in each cell

import matplotlib.pyplot as plt
import seaborn as sns
# Set seaborn style
sns.set_style("whitegrid")

import warnings
warnings.filterwarnings("ignore")

In [4]:
# File path
file_path = "/content/drive/MyDrive/Advanced-Projects/Data/Patient_Data_Processed.csv"
file_path_md = "/content/drive/MyDrive/Advanced-Projects/Data/Medicine_Data.csv"
# Load CSV into DataFrame
patient_data = pd.read_csv(file_path)
meds_data = pd.read_csv(file_path_md)

In [5]:
display(patient_data.head(), meds_data.head())

,Unnamed: 0,inpatient.number,DestinationDischarge,admission.ward,admission.way,discharge.department,visit.times,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,weight,height,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,Killip.grade,myocardial.infarction,congestive.heart.failure,peripheral.vascular.disease,cerebrovascular.disease,dementia,Chronic.obstructive.pulmonary.disease,connective.tissue.disease,diabetes,hemiplegia,leukemia,malignant.lymphoma,solid.tumor,AIDS,type.II.respiratory.failure,consciousness,eye.opening,verbal.response,movement,oxygen.inhalation,fio2,acute.renal.failure,outcome.during.hospitalization,death.within.28.days,re.admission.within.28.days,death.within.3.months,re.admission.within.3.months,death.within.6.months,re.admission.within.6.months,GCS,dischargeDay,ageCat,mean.corpuscular.volume,hemoglobin,mean.hemoglobin.volume,hematocrit,occupation,neutrophil.count,neutrophil.ratio,platelet,eosinophil.count,eosinophil.ratio,basophil.count,basophil.ratio,mean.hemoglobin.concentration,lymphocyte.count,red.blood.cell,monocyte.count,monocyte.ratio,white.blood.cell,uric.acid,urea,creatinine.enzymatic.method,carbon.dioxide.binding.capacity,chloride,potassium,calcium,sodium,CCI.score,moderate.to.severe.chronic.kidney.disease,peptic.ulcer.disease,liver.disease,return.to.emergency.department.within.6.months
0,1,857781,Home,Cardiology,NonEmergency,Cardiology,1,Male,36.7,87,19,102,64,76.666667,50.0,1.64,18.590125,Both,III,III,0,0,0,0,0,1,0,1,0,0,0,0,0,NonTypeII,Clear,4,5,6,OxygenTherapy,33,0,Alive,0,0,0,0,0,0,15,11,"(69,79]",94.5,131.0,32.0,0.387,UrbanResident,7.05,0.746,102.0,0.02,0.002,0.05,0.006,338.0,1.51,4.09,0.80,0.085,9.43,685.0,12.55,108.3,21.6,101.9,5.59,2.28,134.6,2.0,0.0,0.0,0.0,0.0
1,2,743087,Home,Cardiology,NonEmergency,Cardiology,1,Female,36.8,95,18,150,70,96.666667,51.0,1.63,19.195303,Both,III,I,0,0,0,0,0,0,0,0,0,0,0,0,0,NonTypeII,Clear,4,5,6,OxygenTherapy,33,0,Alive,0,0,0,0,0,0,15,8,"(69,79]",88.6,114.0,28.9,0.349,UrbanResident,3.73,0.702,123.0,0.05,0.009,0.01,0.002,326.0,1.18,3.93,0.35,0.066,5.32,170.0,4.29,62.0,26.5,105.6,3.62,2.28,144.0,0.0,0.0,0.0,0.0,0.0
2,3,866418,Home,Cardiology,NonEmergency,Cardiology,2,Male,36.5,98,18,102,67,78.666667,70.0,1.70,24.221453,Both,II,II,0,0,0,0,0,0,0,0,0,0,0,0,0,NonTypeII,Clear,4,5,6,OxygenTherapy,33,0,Alive,0,0,0,0,0,0,15,5,"(59,69]",96.4,144.0,32.6,0.427,farmer,11.54,0.887,169.0,0.02,0.002,0.03,0.002,339.0,0.75,4.43,0.67,0.051,13.01,567.0,15.99,185.1,21.6,107.0,4.15,2.56,142.2,0.0,0.0,0.0,0.0,0.0
3,4,775928,Home,Cardiology,Emergency,Cardiology,1,Male,36.0,73,19,110,74,86.000000,65.0,1.70,22.491349,Both,III,II,0,0,0,0,0,1,0,0,0,0,0,0,0,NonTypeII,Clear,4,5,6,OxygenTherapy,33,0,Alive,0,1,0,1,0,1,15,11,"(69,79]",102.2,94.0,34.7,0.277,UrbanResident,1.22,0.559,35.0,0.06,0.028,0.00,0.000,339.0,0.71,2.71,0.19,0.087,2.18,635.0,8.16,104.8,21.1,101.6,3.76,2.35,136.3,2.0,1.0,0.0,0.0,1.0
4,5,810128,Home,Cardiology,NonEmergency,Cardiology,1,Female,35.0,88,19,134,62,86.000000,76.0,1.55,31.633715,Both,III,II,0,0,0,0,0,0,0,0,0,0,0,0,0,NonTypeII,Clear,4,5,6,OxygenTherapy,33,0,Alive,0,0,0,0,0,0,15,5,"(69,79]",90.0,120.0,28.6,0.378,UrbanResident,4.28,0.697,104.0,0.05,0.008,0.01,0.002,317.0,1.27,4.20,0.53,0.086,6.14,432.0,6.86,83.9,22.8,104.3,3.94,2.42,136.6,0.0,0.0,0.0,0.0,0.0


,Unnamed: 0,inpatient.number,Drug_name
0,1,857781,sulfotanshinone sodium injection
1,2,857781,Furosemide tablet
2,3,857781,Enoxaparin Sodium injection
3,4,857781,Meglumine Adenosine Cyclophosphate for injection
4,5,857781,Furosemide injection


In [6]:
# Add medication count data

meds_count = meds_data.groupby('inpatient.number').size().reset_index(name='med_count')
patient_data = patient_data.merge(meds_count, on='inpatient.number', how='left').fillna({'med_count': 0})
patient_data['med_count'] = patient_data['med_count'].astype(int)

In [7]:
label = patient_data['re.admission.within.28.days']
patient_data.drop(['Unnamed: 0','inpatient.number','re.admission.within.28.days','re.admission.within.3.months','re.admission.within.6.months','return.to.emergency.department.within.6.months'],axis=1,inplace=True)

# Encoding Non-numeric Variables

In [8]:
from sklearn.preprocessing import LabelEncoder

# Identify columns with data type 'object'
categorical_cols = patient_data.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical Columns to Encode: {categorical_cols}")

# Initialize LabelEncoder and Encode
label_encoders = {}  # To store encoders if needed later
for col in categorical_cols:
    le = LabelEncoder()
    patient_data[col] = le.fit_transform(patient_data[col].astype(str))
    label_encoders[col] = le  # Save encoder if needed later for inverse_transform

print("Label Encoding Completed!")


Categorical Columns to Encode: ['DestinationDischarge', 'admission.ward', 'admission.way', 'discharge.department', 'gender', 'type.of.heart.failure', 'NYHA.cardiac.function.classification', 'Killip.grade', 'type.II.respiratory.failure', 'consciousness', 'oxygen.inhalation', 'outcome.during.hospitalization', 'ageCat', 'occupation']
Label Encoding Completed!


# Remove Low Variance Features

In [9]:
from sklearn.feature_selection import VarianceThreshold

# Set threshold (e.g., 0.01 means a feature needs at least 1% variance to be kept)
threshold = 0.01

# Initialize VarianceThreshold
var_thresh = VarianceThreshold(threshold=threshold)

# Fit and transform
filtered_data = var_thresh.fit_transform(patient_data)

# Get selected features (those that remain)
selected_features = patient_data.columns[var_thresh.get_support()]

# Identify removed features
removed_features = patient_data.columns[~patient_data.columns.isin(selected_features)]

# Convert the filtered data back to a DataFrame
patient_data_low_variance = pd.DataFrame(filtered_data, columns=selected_features)

# Display results
print(f"Original Number of Features: {patient_data.shape[1]}")
print(f"Number of Features Removed: {len(removed_features)}")
print(f"Removed Features: {removed_features.tolist()}")
print(f"Remaining Features: {selected_features.tolist()}")
print(f"Number of Features Remaining: {len(patient_data_low_variance.columns.tolist())}")


Original Number of Features: 77
Number of Features Removed: 12
Removed Features: ['height', 'connective.tissue.disease', 'hemiplegia', 'leukemia', 'malignant.lymphoma', 'AIDS', 'acute.renal.failure', 'hematocrit', 'eosinophil.ratio', 'basophil.count', 'basophil.ratio', 'monocyte.ratio']
Remaining Features: ['DestinationDischarge', 'admission.ward', 'admission.way', 'discharge.department', 'visit.times', 'gender', 'body.temperature', 'pulse', 'respiration', 'systolic.blood.pressure', 'diastolic.blood.pressure', 'map', 'weight', 'BMI', 'type.of.heart.failure', 'NYHA.cardiac.function.classification', 'Killip.grade', 'myocardial.infarction', 'congestive.heart.failure', 'peripheral.vascular.disease', 'cerebrovascular.disease', 'dementia', 'Chronic.obstructive.pulmonary.disease', 'diabetes', 'solid.tumor', 'type.II.respiratory.failure', 'consciousness', 'eye.opening', 'verbal.response', 'movement', 'oxygen.inhalation', 'fio2', 'outcome.during.hospitalization', 'death.within.28.days', 'death.

# Remove Correlated Features

In [10]:
import numpy as np

# Set correlation threshold
corr_threshold = 0.7

# Compute correlation matrix
corr_matrix = patient_data_low_variance.corr().abs()

# Select upper triangle of correlation matrix
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identify features with correlation greater than threshold
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > corr_threshold)]

# Drop the correlated features
patient_data_uncorrelated = patient_data_low_variance.drop(columns=to_drop)

# Display results
original_features_count = patient_data_low_variance.shape[1]
removed_features = to_drop
selected_features = patient_data_uncorrelated.columns.tolist()

print(f"Original Number of Features: {original_features_count}")
print(f"Number of Features Removed: {len(removed_features)}")
print(f"Removed Features: {removed_features}")
print(f"Remaining Features: {selected_features}")
print(f"Number of Features Remaining: {len(patient_data_uncorrelated.columns.tolist())}")


Original Number of Features: 65
Number of Features Removed: 10
Removed Features: ['map', 'verbal.response', 'movement', 'death.within.3.months', 'death.within.6.months', 'GCS', 'mean.hemoglobin.volume', 'red.blood.cell', 'white.blood.cell', 'sodium']
Remaining Features: ['DestinationDischarge', 'admission.ward', 'admission.way', 'discharge.department', 'visit.times', 'gender', 'body.temperature', 'pulse', 'respiration', 'systolic.blood.pressure', 'diastolic.blood.pressure', 'weight', 'BMI', 'type.of.heart.failure', 'NYHA.cardiac.function.classification', 'Killip.grade', 'myocardial.infarction', 'congestive.heart.failure', 'peripheral.vascular.disease', 'cerebrovascular.disease', 'dementia', 'Chronic.obstructive.pulmonary.disease', 'diabetes', 'solid.tumor', 'type.II.respiratory.failure', 'consciousness', 'eye.opening', 'oxygen.inhalation', 'fio2', 'outcome.during.hospitalization', 'death.within.28.days', 'dischargeDay', 'ageCat', 'mean.corpuscular.volume', 'hemoglobin', 'occupation', '

# SHAP (Feature Importance Analysis)

In [11]:
import xgboost as xgb
import shap

# Train XGBoost model
X = patient_data_uncorrelated
y = label

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X, y)

# SHAP Explainer
explainer = shap.Explainer(model, X)
shap_values = explainer(X)


In [12]:

# Mean absolute SHAP values → Global importance
shap_importance = pd.DataFrame({
    'Feature': X.columns,
    'SHAP_Value': np.abs(shap_values.values).mean(axis=0)
})

shap_importance = shap_importance.sort_values(by='SHAP_Value', ascending=False)
print(shap_importance)

                                      Feature  SHAP_Value
47                                   chloride    0.534559
37                           neutrophil.ratio    0.341659
44                                       urea    0.296871
48                                  potassium    0.296720
45                creatinine.enzymatic.method    0.285082
38                                   platelet    0.282247
54                                  med_count    0.280350
49                                    calcium    0.277599
43                                  uric.acid    0.271837
13                      type.of.heart.failure    0.270625
39                           eosinophil.count    0.268359
12                                        BMI    0.235100
41                           lymphocyte.count    0.232646
34                                 hemoglobin    0.232537
36                           neutrophil.count    0.232303
14       NYHA.cardiac.function.classification    0.230371
7             

In [15]:
# Set SHAP threshold
threshold = 0.1

# Select important features
important_features = shap_importance[shap_importance['SHAP_Value'] > threshold]['Feature'].tolist()

# Store original feature count before dropping
original_features_count = patient_data_uncorrelated.shape[1]

# Identify removed features
removed_features = patient_data_uncorrelated.columns[~patient_data_uncorrelated.columns.isin(important_features)].tolist()

# Create new DataFrame with only important features
patient_data_shap_filtered = patient_data_uncorrelated[important_features]

# Get the selected features list
selected_features = patient_data_shap_filtered.columns.tolist()

# Display results
print(f"Original Number of Features: {original_features_count}")
print(f"Number of Features Removed: {len(removed_features)}")
print(f"Removed Features: {removed_features}")
print(f"Remaining Features: {selected_features}")
print(f"Number of Features Remaining: {len(selected_features)}")

Original Number of Features: 55
Number of Features Removed: 25
Removed Features: ['admission.ward', 'admission.way', 'discharge.department', 'visit.times', 'gender', 'respiration', 'myocardial.infarction', 'congestive.heart.failure', 'peripheral.vascular.disease', 'cerebrovascular.disease', 'dementia', 'Chronic.obstructive.pulmonary.disease', 'solid.tumor', 'type.II.respiratory.failure', 'consciousness', 'eye.opening', 'oxygen.inhalation', 'fio2', 'outcome.during.hospitalization', 'death.within.28.days', 'ageCat', 'CCI.score', 'moderate.to.severe.chronic.kidney.disease', 'peptic.ulcer.disease', 'liver.disease']
Remaining Features: ['chloride', 'neutrophil.ratio', 'urea', 'potassium', 'creatinine.enzymatic.method', 'platelet', 'med_count', 'calcium', 'uric.acid', 'type.of.heart.failure', 'eosinophil.count', 'BMI', 'lymphocyte.count', 'hemoglobin', 'neutrophil.count', 'NYHA.cardiac.function.classification', 'pulse', 'weight', 'mean.corpuscular.volume', 'diastolic.blood.pressure', 'systol

# Random Forest (Feature Importance Analysis)

In [14]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Use filtered or unfiltered data depending on your stage
X_rf = patient_data_uncorrelated  # Or patient_data_shap_filtered if you want to check the refined set
y_rf = label

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_rf, y_rf)

# Feature Importance
rf_importance = pd.DataFrame({
    'Feature': X_rf.columns,
    'Importance': rf_model.feature_importances_
})

rf_importance = rf_importance.sort_values(by='Importance', ascending=False)

print(rf_importance)

                                      Feature  Importance
45                creatinine.enzymatic.method    0.049537
38                                   platelet    0.048655
43                                  uric.acid    0.042951
33                    mean.corpuscular.volume    0.041336
47                                   chloride    0.040779
44                                       urea    0.040147
12                                        BMI    0.039157
40              mean.hemoglobin.concentration    0.039086
39                           eosinophil.count    0.038562
7                                       pulse    0.036533
37                           neutrophil.ratio    0.034668
36                           neutrophil.count    0.034479
9                     systolic.blood.pressure    0.034326
41                           lymphocyte.count    0.033496
11                                     weight    0.032768
48                                  potassium    0.032754
10            

In [16]:
# Selecting Top 30 Features from Random Forest - Feature Importance

rf_top_30_features = rf_importance['Feature'].head(30).tolist()
patient_data_rf_filtered = patient_data_uncorrelated[rf_top_30_features]


In [17]:
display(patient_data_rf_filtered.head(), patient_data_shap_filtered.head())

,creatinine.enzymatic.method,platelet,uric.acid,mean.corpuscular.volume,chloride,urea,BMI,mean.hemoglobin.concentration,eosinophil.count,pulse,neutrophil.ratio,neutrophil.count,systolic.blood.pressure,lymphocyte.count,weight,potassium,diastolic.blood.pressure,calcium,carbon.dioxide.binding.capacity,hemoglobin,monocyte.count,body.temperature,dischargeDay,CCI.score,med_count,ageCat,respiration,NYHA.cardiac.function.classification,Killip.grade,DestinationDischarge
0,108.3,102.0,685.0,94.5,101.9,12.55,18.590125,338.0,0.02,87.0,0.746,7.05,102.0,1.51,50.0,5.59,64.0,2.28,21.6,131.0,0.80,36.7,11.0,2.0,13.0,5.0,19.0,1.0,2.0,2.0
1,62.0,123.0,170.0,88.6,105.6,4.29,19.195303,326.0,0.05,95.0,0.702,3.73,150.0,1.18,51.0,3.62,70.0,2.28,26.5,114.0,0.35,36.8,8.0,0.0,3.0,5.0,18.0,1.0,0.0,2.0
2,185.1,169.0,567.0,96.4,107.0,15.99,24.221453,339.0,0.02,98.0,0.887,11.54,102.0,0.75,70.0,4.15,67.0,2.56,21.6,144.0,0.67,36.5,5.0,0.0,7.0,4.0,18.0,0.0,1.0,2.0
3,104.8,35.0,635.0,102.2,101.6,8.16,22.491349,339.0,0.06,73.0,0.559,1.22,110.0,0.71,65.0,3.76,74.0,2.35,21.1,94.0,0.19,36.0,11.0,2.0,8.0,5.0,19.0,1.0,1.0,2.0
4,83.9,104.0,432.0,90.0,104.3,6.86,31.633715,317.0,0.05,88.0,0.697,4.28,134.0,1.27,76.0,3.94,62.0,2.42,22.8,120.0,0.53,35.0,5.0,0.0,5.0,5.0,19.0,1.0,1.0,2.0


,chloride,neutrophil.ratio,urea,potassium,creatinine.enzymatic.method,platelet,med_count,calcium,uric.acid,type.of.heart.failure,eosinophil.count,BMI,lymphocyte.count,hemoglobin,neutrophil.count,NYHA.cardiac.function.classification,pulse,weight,mean.corpuscular.volume,diastolic.blood.pressure,systolic.blood.pressure,dischargeDay,occupation,DestinationDischarge,mean.hemoglobin.concentration,Killip.grade,body.temperature,monocyte.count,diabetes,carbon.dioxide.binding.capacity
0,101.9,0.746,12.55,5.59,108.3,102.0,13.0,2.28,685.0,0.0,0.02,18.590125,1.51,131.0,7.05,1.0,87.0,50.0,94.5,64.0,102.0,11.0,2.0,2.0,338.0,2.0,36.7,0.80,1.0,21.6
1,105.6,0.702,4.29,3.62,62.0,123.0,3.0,2.28,170.0,0.0,0.05,19.195303,1.18,114.0,3.73,1.0,95.0,51.0,88.6,70.0,150.0,8.0,2.0,2.0,326.0,0.0,36.8,0.35,0.0,26.5
2,107.0,0.887,15.99,4.15,185.1,169.0,7.0,2.56,567.0,0.0,0.02,24.221453,0.75,144.0,11.54,0.0,98.0,70.0,96.4,67.0,102.0,5.0,3.0,2.0,339.0,1.0,36.5,0.67,0.0,21.6
3,101.6,0.559,8.16,3.76,104.8,35.0,8.0,2.35,635.0,0.0,0.06,22.491349,0.71,94.0,1.22,1.0,73.0,65.0,102.2,74.0,110.0,11.0,2.0,2.0,339.0,1.0,36.0,0.19,0.0,21.1
4,104.3,0.697,6.86,3.94,83.9,104.0,5.0,2.42,432.0,0.0,0.05,31.633715,1.27,120.0,4.28,1.0,88.0,76.0,90.0,62.0,134.0,5.0,2.0,2.0,317.0,1.0,35.0,0.53,0.0,22.8


# Scaling and Splitting for SHAP Features (Best for Logistic, SVM, KNN)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Final SHAP-selected 30 features dataset
X_shap = patient_data_shap_filtered
y = label

# Train-Test Split
X_shap_train, X_shap_test, y_train, y_test = train_test_split(X_shap, y, test_size=0.2, random_state=42)

# Scaling for models that need it (Logistic Regression, SVM, KNN)
scaler = StandardScaler()
X_shap_train_scaled = scaler.fit_transform(X_shap_train)
X_shap_test_scaled = scaler.transform(X_shap_test)

print("SHAP Data Split and Scaled Successfully!")
print(f"Train Shape: {X_shap_train_scaled.shape}, Test Shape: {X_shap_test_scaled.shape}")


SHAP Data Split and Scaled Successfully!
Train Shape: (1606, 30), Test Shape: (402, 30)


# Splitting for Tree-Based Models (RF, XGBoost, LightGBM)

In [19]:
# Final RF-selected 30 features dataset (or SHAP features if you want to use the same for tree models)
X_rf = patient_data_rf_filtered  # or X_shap if you prefer SHAP for trees as well
y = label

# Train-Test Split (No Scaling Needed)
X_rf_train, X_rf_test, y_train_rf, y_test_rf = train_test_split(X_rf, y, test_size=0.2, random_state=42)

print("RF Data Split Successfully!")
print(f"Train Shape: {X_rf_train.shape}, Test Shape: {X_rf_test.shape}")


RF Data Split Successfully!
Train Shape: (1606, 30), Test Shape: (402, 30)


# Model Training

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from joblib import Parallel, delayed
import numpy as np

# SHAP Scaled Features
X_shap_train_scaled, X_shap_test_scaled, y_train, y_test = X_shap_train_scaled, X_shap_test_scaled, y_train, y_test

# RF Raw Features
X_rf_train, X_rf_test, y_train_rf, y_test_rf = X_rf_train, X_rf_test, y_train, y_test

# Set up models
models_shap_scaled = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVM': SVC(kernel='linear', probability=True),
    'KNN': KNeighborsClassifier()
}

models_rf_raw = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', tree_method='hist'),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1)
}

# Evaluation function using Joblib for parallel CV
def evaluate_model(model, X, y, model_name):
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    mean_score = np.mean(scores)
    print(f"{model_name} - Mean Accuracy: {mean_score:.4f}")
    return model_name, mean_score

# Parallel execution for SHAP Scaled Models
print("Evaluating Models (SHAP Features - Scaled)")
shap_results = Parallel(n_jobs=-1)(delayed(evaluate_model)(model, X_shap_train_scaled, y_train, name)
                                   for name, model in models_shap_scaled.items())

# Parallel execution for RF Raw Models
print("\nEvaluating Models (RF Features - Raw)")
rf_results = Parallel(n_jobs=-1)(delayed(evaluate_model)(model, X_rf_train, y_train_rf, name)
                                 for name, model in models_rf_raw.items())


Evaluating Models (SHAP Features - Scaled)

Evaluating Models (RF Features - Raw)


In [31]:
# Convert SHAP CV Results to DataFrame
shap_results_df = pd.DataFrame(list(shap_results.items()), columns=['Model', 'CrossVal_Accuracy'])
shap_results_df['Feature_Set'] = 'SHAP Scaled'

# Convert RF CV Results to DataFrame
rf_results_df = pd.DataFrame(list(rf_results.items()), columns=['Model', 'CrossVal_Accuracy'])
rf_results_df['Feature_Set'] = 'RF Raw'

# Combine into One DataFrame
cv_results_df = pd.concat([shap_results_df, rf_results_df], ignore_index=True)

# Display the DataFrame
cv_results_df_sorted = cv_results_df.sort_values(by='CrossVal_Accuracy')
cv_results_df_sorted_desc = cv_results_df.sort_values(by='CrossVal_Accuracy', ascending=False)
cv_results_df_sorted_desc

,Model,CrossVal_Accuracy,Feature_Set
1,SVM,0.935243,SHAP Scaled
3,RandomForest,0.935243,RF Raw
4,XGBoost,0.935241,RF Raw
5,LightGBM,0.934620,RF Raw
0,LogisticRegression,0.933995,SHAP Scaled
2,KNN,0.930882,SHAP Scaled


# Model Evaluation

In [26]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
import pandas as pd

results = []

# Evaluate SHAP Scaled Models
models_shap_scaled = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVM': SVC(kernel='linear', probability=True),
    'KNN': KNeighborsClassifier()
}

for name, model in models_shap_scaled.items():
    model.fit(X_shap_train_scaled, y_train)
    y_pred = model.predict(X_shap_test_scaled)
    y_prob = model.predict_proba(X_shap_test_scaled)[:, 1]

    results.append([name, 'SHAP Scaled',
                    accuracy_score(y_test, y_pred),
                    roc_auc_score(y_test, y_prob),
                    f1_score(y_test, y_pred),
                    precision_score(y_test, y_pred),
                    recall_score(y_test, y_pred)])

# Evaluate RF Raw Models
models_rf_raw = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', tree_method='hist'),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1)
}

for name, model in models_rf_raw.items():
    model.fit(X_rf_train, y_train_rf)
    y_pred = model.predict(X_rf_test)
    y_prob = model.predict_proba(X_rf_test)[:, 1]

    results.append([name, 'RF Raw',
                    accuracy_score(y_test_rf, y_pred),
                    roc_auc_score(y_test_rf, y_prob),
                    f1_score(y_test_rf, y_pred),
                    precision_score(y_test_rf, y_pred),
                    recall_score(y_test_rf, y_pred)])



[LightGBM] [Info] Number of positive: 104, number of negative: 1502
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000538 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3837
[LightGBM] [Info] Number of data points in the train set: 1606, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.064757 -> initscore=-2.670162
[LightGBM] [Info] Start training from score -2.670162


In [34]:
# Convert to DataFrame
results_df = pd.DataFrame(results, columns=['Model', 'Feature Set', 'Accuracy', 'AUC', 'F1-Score', 'Precision', 'Recall'])
results_df_sorted_desc = results_df.sort_values(by=['Accuracy', 'AUC'], ascending=[False, False])
results_df_sorted_desc

,Model,Feature Set,Accuracy,AUC,F1-Score,Precision,Recall
0,LogisticRegression,SHAP Scaled,0.910448,0.683212,0.00,0.00,0.000000
5,LightGBM,RF Raw,0.910448,0.626063,0.00,0.00,0.000000
3,RandomForest,RF Raw,0.910448,0.604964,0.00,0.00,0.000000
4,XGBoost,RF Raw,0.910448,0.599423,0.00,0.00,0.000000
1,SVM,SHAP Scaled,0.910448,0.450137,0.00,0.00,0.000000
2,KNN,SHAP Scaled,0.905473,0.521554,0.05,0.25,0.027778


# Address Class Imbalance
**Key Observations from Your Results:**
* Accuracy is misleading (~91%) – likely because most patients are NOT readmitted.
* F1, Precision, Recall = 0 or very low → Your models are predicting almost NO readmissions.
* AUC is low (~0.6) → Poor separability between readmitted and non-readmitted patients.

**Why: Readmission is often a rare event. If only 9% are readmitted, predicting "NO" for all gives 91% accuracy, but that's useless.**

### **Solutions:**
Resampling Techniques:
* SMOTE (Synthetic Minority Over-sampling Technique) – Create synthetic samples.
* Undersampling – Reduce majority class samples.
* Combine Both – SMOTE + Undersampling (common in healthcare).

In [35]:
from imblearn.over_sampling import SMOTE


# SHAP Scaled Features
X_shap_train_scaled, X_shap_test_scaled, y_train, y_test = X_shap_train_scaled, X_shap_test_scaled, y_train, y_test

# RF Raw Features
X_rf_train, X_rf_test, y_train_rf, y_test_rf = X_rf_train, X_rf_test, y_train, y_test


# Apply SMOTE Resampling
smote = SMOTE(random_state=42)

X_shap_train_resampled, y_shap_train_resampled = smote.fit_resample(X_shap_train_scaled, y_train)
X_rf_train_resampled, y_rf_train_resampled = smote.fit_resample(X_rf_train, y_train_rf)


# Set up models
models_shap_scaled = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVM': SVC(kernel='linear', probability=True),
    'KNN': KNeighborsClassifier()
}

models_rf_raw = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', tree_method='hist'),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1)
}


In [37]:

# -------------------------------------------
# PART 1: CROSS-VALIDATION (With SMOTE Resampled Data)
# -------------------------------------------

# Evaluation function using Joblib for parallel CV
def evaluate_model(model, X, y, model_name):
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    mean_score = np.mean(scores)
    print(f"{model_name} - Mean Accuracy: {mean_score:.4f}")
    return model_name, mean_score

# Parallel execution for SHAP Scaled Models
print("Evaluating Models (SHAP Features - Scaled) with SMOTE")
shap_results_smote = Parallel(n_jobs=-1)(delayed(evaluate_model)(model, X_shap_train_resampled, y_shap_train_resampled, name)
                                         for name, model in models_shap_scaled.items())

# Parallel execution for RF Raw Models
print("\nEvaluating Models (RF Features - Raw) with SMOTE")
rf_results_smote = Parallel(n_jobs=-1)(delayed(evaluate_model)(model, X_rf_train_resampled, y_rf_train_resampled, name)
                                       for name, model in models_rf_raw.items())

# Convert CV Results to DataFrame
shap_results_df_smote = pd.DataFrame(shap_results_smote, columns=['Model', 'CrossVal_Accuracy'])
shap_results_df_smote['Feature_Set'] = 'SHAP Scaled (SMOTE)'

rf_results_df_smote = pd.DataFrame(rf_results_smote, columns=['Model', 'CrossVal_Accuracy'])
rf_results_df_smote['Feature_Set'] = 'RF Raw (SMOTE)'

cv_results_smote_df = pd.concat([shap_results_df_smote, rf_results_df_smote], ignore_index=True)
cv_results_smote_df_sorted = cv_results_smote_df.sort_values(by='CrossVal_Accuracy', ascending=False)

print("\nCross-Validation Results with SMOTE:")

cv_results_smote_df_sorted

Evaluating Models (SHAP Features - Scaled) with SMOTE

Evaluating Models (RF Features - Raw) with SMOTE

Cross-Validation Results with SMOTE:


,Model,CrossVal_Accuracy,Feature_Set
3,RandomForest,0.977702,RF Raw (SMOTE)
5,LightGBM,0.965390,RF Raw (SMOTE)
4,XGBoost,0.965057,RF Raw (SMOTE)
2,KNN,0.773975,SHAP Scaled (SMOTE)
1,SVM,0.681093,SHAP Scaled (SMOTE)
0,LogisticRegression,0.680430,SHAP Scaled (SMOTE)


In [38]:

# -------------------------------------------
# PART 2: EVALUATION ON TEST SET (After Resampling for Training)
# -------------------------------------------

test_results = []

# Evaluate SHAP Scaled Models (Test Set)
for name, model in models_shap_scaled.items():
    model.fit(X_shap_train_resampled, y_shap_train_resampled)
    y_pred = model.predict(X_shap_test_scaled)
    y_prob = model.predict_proba(X_shap_test_scaled)[:, 1]

    test_results.append([name, 'SHAP Scaled (SMOTE)',
                         accuracy_score(y_test, y_pred),
                         roc_auc_score(y_test, y_prob),
                         f1_score(y_test, y_pred),
                         precision_score(y_test, y_pred),
                         recall_score(y_test, y_pred)])

# Evaluate RF Raw Models (Test Set)
for name, model in models_rf_raw.items():
    model.fit(X_rf_train_resampled, y_rf_train_resampled)
    y_pred = model.predict(X_rf_test)
    y_prob = model.predict_proba(X_rf_test)[:, 1]

    test_results.append([name, 'RF Raw (SMOTE)',
                         accuracy_score(y_test_rf, y_pred),
                         roc_auc_score(y_test_rf, y_prob),
                         f1_score(y_test_rf, y_pred),
                         precision_score(y_test_rf, y_pred),
                         recall_score(y_test_rf, y_pred)])

# Convert Test Results to DataFrame
test_results_df = pd.DataFrame(test_results, columns=['Model', 'Feature Set', 'Accuracy', 'AUC', 'F1-Score', 'Precision', 'Recall'])
test_results_df_sorted = test_results_df.sort_values(by=['Accuracy', 'AUC'], ascending=[False, False])

print("\nTest Set Evaluation Results with SMOTE:")
test_results_df_sorted


[LightGBM] [Info] Number of positive: 1502, number of negative: 1502
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001320 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7642
[LightGBM] [Info] Number of data points in the train set: 3004, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Test Set Evaluation Results with SMOTE:


,Model,Feature Set,Accuracy,AUC,F1-Score,Precision,Recall
4,XGBoost,RF Raw (SMOTE),0.912935,0.656117,0.102564,0.666667,0.055556
5,LightGBM,RF Raw (SMOTE),0.907960,0.651488,0.097561,0.400000,0.055556
3,RandomForest,RF Raw (SMOTE),0.907960,0.589481,0.051282,0.333333,0.027778
0,LogisticRegression,SHAP Scaled (SMOTE),0.691542,0.677216,0.243902,0.156250,0.555556
1,SVM,SHAP Scaled (SMOTE),0.684080,0.656383,0.248521,0.157895,0.583333
2,KNN,SHAP Scaled (SMOTE),0.577114,0.564056,0.182692,0.110465,0.527778


## Why Was SMOTE Used & What Problem Did It Solve?

#### 1️⃣ Problem Before SMOTE: Class Imbalance
* Readmission within 28 days is a rare event in healthcare data.
* In your dataset, only ~9% patients are readmitted.
* If a model predicts "NO" for all, it would still be 91% accurate, but that’s useless.
* Key Issue: The model learns to focus on the majority class (NO Readmission) and ignores the minority class (Readmission).

#### 2️⃣ What Does SMOTE Do?
* SMOTE (Synthetic Minority Over-sampling Technique) creates synthetic samples for the minority class (readmissions) instead of duplicating existing ones.
* How: It interpolates between real samples, generating artificial "new" samples that resemble existing minority class samples.
* This balances the data → equal number of "Readmitted" and "Not Readmitted" samples.

#### ✅ Why High Recall Was Important (Short Answer):

In readmission prediction (healthcare), high recall is crucial because:

* Recall = Ability to catch all actual readmissions.
* Missing a readmission (low recall) → Patient risk increases → Potential health complications or death.
* False positives (predicting readmission when not needed) → Extra checks or follow-up (less harmful than missing a critical patient).
* In healthcare, it’s safer to over-alert than to miss a high-risk patient.

**Recall matters more than accuracy because missing a readmission can harm the patient.**